In [1]:
# define the model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate, GlobalAveragePooling2D, Activation, Add, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)


In [2]:
img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

shared_fc_layer = Sequential([
    Dense(256, activation='relu', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    #Dense(256, activation='relu', input_shape=(2048, )),
    #BatchNormalization(),
    #Dropout(0.5),
    Dense(1),
])

encoded_a = shared_fc_layer(img_feature_a)
encoded_b = shared_fc_layer(img_feature_b)

negated_encoded_b = Lambda(lambda x: -x, output_shape=(1,))(encoded_b)
diff = Add()([encoded_a, negated_encoded_b])

output = Activation("sigmoid")(diff)

rank_model = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

rank_model.load_weights("./data/ranknet/ranknet.h5")

##########################################################################################################################

base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x) # add a global spatial average pooling layer
inception_layer = Model(inputs=base_model.input, outputs=x)

img_a = Input(shape=(3, 299, 299))
img_b = Input(shape=(3, 299, 299))

img_feature_a = inception_layer(img_a)
img_feature_b = inception_layer(img_b)

output = rank_model([img_feature_a, img_feature_b])

model = Model(inputs=[img_a, img_b], outputs=output)

#for layer in base_model.layers[:249]:
#    layer.trainable = False
#for layer in base_model.layers[249:]:
#    layer.trainable = True

#optimizer = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 3, 299, 299)   0                                            
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 3, 299, 299)   0                                            
____________________________________________________________________________________________________
model_2 (Model)                  (None, 2048)          21802784    input_4[0][0]                    
                                                                   input_5[0][0]                    
____________________________________________________________________________________________________
model_1 (Model)                  (None, 1)             525825      model_2[1][0]           

In [3]:
img_path = "../data/img/"
def img_preprocess(imgid, target_size=(299, 299)):
    try:
        filename = img_path+"%s.jpg"%imgid
        img = image.load_img(filename, target_size=target_size)
        x = image.img_to_array(img)
        x = preprocess_input(x)
        return x
    except Exception, e:
        print str(e)
        return None

In [ ]:
# RUN ONLY ONCE
import cPickle as pickle
import random
import os
import shutil

train_list = pickle.load(open("./data/ranknet/train.list", 'rb'))
valid_list = pickle.load(open("./data/ranknet/valid.list", 'rb'))

if os.path.exists("./data/ranknet_ft"):
    shutil.rmtree("./data/ranknet_ft")
os.makedirs("./data/ranknet_ft/train/none")
os.makedirs("./data/ranknet_ft/valid/none")

white_set = set()
def touch_files(dataset, path, count=-1):
    if count != -1:
        subdata = random.sample(dataset, count)
    else:
        subdata = dataset
    for imgA, sA, imgB, sB, cmpret in subdata:
        if imgA in white_set or img_preprocess(imgA) is not None:
            white_set.add(imgA)
        if imgB in white_set or img_preprocess(imgB) is not None:
            white_set.add(imgB)
        if imgA in white_set and imgB in white_set:
            open(path+"/%s_%s_%d.jpg"%(imgA, imgB, cmpret), 'w').close()
        
touch_files(train_list, path="./data/ranknet_ft/train/none/", count=100000)
touch_files(valid_list, path="./data/ranknet_ft/valid/none/", count=-1)

In [4]:
from keras.preprocessing.image import *
import os

# 自定义DirectoryIterator类，可以返回自定义的label
class CustomDirectoryIterator(DirectoryIterator):  
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x1 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_x2 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, ) , dtype=K.floatx())
        # build batch of image data
        for i, j in enumerate(index_array):
            fname =  self.filenames[j]
            fname = os.path.basename(fname)
            fname, _ = os.path.splitext(fname)
            imgA, imgB, cmpret = fname.split("_")
            x1 = img_preprocess(imgA, self.target_size)
            x2 = img_preprocess(imgB, self.target_size)
            batch_x1[i] = x1
            batch_x2[i] = x2
            batch_y[i] = int(cmpret)
        return [batch_x1, batch_x2], batch_y

# 定义批处理的数据集大小：较小的batch_size可以增加权重调整的次数，同时节省内存的开销
batch_size = 16 

# 图片预处理工具类
IDG = ImageDataGenerator()

# 从目录文件中流式读取数据，避免训练中一次性加载爆内存
train_batch = CustomDirectoryIterator("./data/ranknet_ft/train/", IDG, 
                                      target_size=(299, 299), batch_size=batch_size, shuffle=True)
valid_batch = CustomDirectoryIterator("./data/ranknet_ft/valid/", IDG, 
                                      target_size=(299, 299), batch_size=batch_size, shuffle=True)

Found 99530 images belonging to 1 classes.
Found 4334 images belonging to 1 classes.


In [5]:
model.fit_generator(train_batch, steps_per_epoch=train_batch.samples // batch_size, epochs=5,
                       validation_data=valid_batch, validation_steps=valid_batch.samples // batch_size)

Epoch 1/5
6220/6220 [==============================] - 8619s - loss: 0.6473 - acc: 0.6237 - val_loss: 0.6466 - val_acc: 0.6229
Epoch 2/5
6220/6220 [==============================] - 8627s - loss: 0.5946 - acc: 0.6821 - val_loss: 0.6746 - val_acc: 0.6179
Epoch 3/5
6220/6220 [==============================] - 8634s - loss: 0.5192 - acc: 0.7440 - val_loss: 0.7319 - val_acc: 0.6070
Epoch 4/5
6220/6220 [==============================] - 8630s - loss: 0.4270 - acc: 0.8037 - val_loss: 0.8088 - val_acc: 0.6123
Epoch 5/5
3324/6220 [===============>..............] - ETA: 3970s - loss: 0.3544 - acc: 0.8457

KeyboardInterrupt: 

In [6]:
model.save_weights("./data/ranknet_ft/ranknet_ft.h5")

In [ ]:
import os
import random
import shutil

path = "./data/finetune/valid/none/"
for fname in os.listdir(path):
    if random.random() < 0.8:
        os.remove(path+fname)

In [ ]:
# 验证模型的初始化与rank model的结果相一致
import os
import cPickle as pickle

Y_predict_by_rank_model = pickle.load(open("./data/valid_predict_by_rank_model.pick", 'rb'))
Y_predict = model.predict_generator(valid_batch, steps=valid_batch.samples // batch_size + 1)

for i, fname in enumerate(valid_batch.filenames):
    fname = os.path.basename(fname)
    fname, _ = os.path.splitext(fname)
    imgA, imgB, _ = fname.split("_")
    if (imgA, imgB) not in Y_predict_by_rank_model:
        continue
    y_r = Y_predict_by_rank_model[(imgA, imgB)]
    y = Y_predict[i, 1]
    print y_r, y

In [ ]:
import lmdb

env = lmdb.open("./data/features")
txn = env.begin()

def testOutput(fname):
    fname = os.path.basename(fname)
    fname, _ = os.path.splitext(fname)
    imgA, imgB, cmpret = fname.split("_")
    
    rrr = model.predict([np.array([x1]), np.array([x2])])
    
    x1 = img_preprocess(imgA)
    x2 = img_preprocess(imgB)
    f1 = inception_layer.predict(np.array([x1]))
    f2 = inception_layer.predict(np.array([x2]))
    r = rank_model.predict([f1, f2])  
    
    fa = txn.get(imgA)
    fb = txn.get(imgB)
    fa = np.fromstring(fa, np.float32)
    fb = np.fromstring(fb, np.float32)
    rr = Y_predict_by_rank_model[(imgA, imgB)]
    return f1, f2, r, fa, fb, rr, rrr

testOutput(valid_batch.filenames[10])